In [1]:
import nltk
import pandas as pd
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [31]:
a=[]
with open('court-V-N.csv','r') as f:
 for line in f:
    line=line.replace(',','')
    line=line.replace('\n','')
    line=line.lower()
    line=line.split()
    a.append(line)
    
        
        

In [32]:
finder  = BigramCollocationFinder.from_documents(a)


In [33]:
finder.nbest(bigram_measures.pmi, 10)


[('арбитраж', 'встретиться'),
 ('армия', 'восстановиться'),
 ('ассоциироватьсь', 'большой'),
 ('аэропорт', 'расторжение'),
 ('балтика', 'нарушить'),
 ('банковский', 'комиссия'),
 ('безоговорочно', 'уступить'),
 ('бессмысленность', 'мероприятие'),
 ('большой', 'политика'),
 ('бригадир', 'виктор')]

Поставим фильтр, чтобы видеть наиболее частотные, например 10:

In [51]:
finder.apply_freq_filter(10)
finder.nbest(bigram_measures.pmi, 10)
pmi = finder.nbest(bigram_measures.pmi, 10)

Теперь получили более вероятную выборку. А если еще больше?

In [52]:
finder.apply_freq_filter(20)
finder.nbest(bigram_measures.pmi, 10)

[('принять', 'решение'),
 ('удовлетворить', 'иск'),
 ('суд', 'рассмотреть'),
 ('суд', 'принять'),
 ('суд', 'отказать'),
 ('суд', 'отклонить'),
 ('суд', 'признать'),
 ('суд', 'удовлетворить'),
 ('суд', 'вынести'),
 ('суд', 'приговорить')]

Отлично! Данные документа не содержат стоп-слов, однако много коллокаций со словом суд. Посмотрим на коллокации без него.

In [53]:
finder.apply_word_filter(lambda w: w in ('суд'))

In [54]:
finder.nbest(bigram_measures.pmi,10)

[('принять', 'решение'), ('удовлетворить', 'иск')]

В итоге получаем совсем мало, толь 2 коллокации. Наиболее привлекательной мне всё же видится выборка с частотностью 10. Рассмотрим по такому же алгоритму коллокации по другой метрике, например, Log-Likelihood.

In [55]:
finder  = BigramCollocationFinder.from_documents(a)
finder.apply_freq_filter(10)
l = finder.nbest(bigram_measures.likelihood_ratio, 10)

Отлично! Как мы видим, выборки практически совпадают. Сравним теперь с золотым стандартом и сделаем выводы.

In [56]:
golden_standart = [
    ('принять', 'решение'),
    ('вынести', 'приговор'), 
    ('удовлетворить', 'иск'),
    ('возбудить', 'дело'), 
    ('проведение', 'следствие'),
    ('наложить', 'арест'),
    ('провести', 'разбирательство'), 
    ('выдать', 'санкция'),
    ('продлить', 'срок'),
    ('удовлетворить', 'ходатайство')
]

In [57]:
from nltk.metrics.spearman import *
print(list(ranks_from_sequence(golden_standart)))

[(('принять', 'решение'), 0), (('вынести', 'приговор'), 1), (('удовлетворить', 'иск'), 2), (('возбудить', 'дело'), 3), (('проведение', 'следствие'), 4), (('наложить', 'арест'), 5), (('провести', 'разбирательство'), 6), (('выдать', 'санкция'), 7), (('продлить', 'срок'), 8), (('удовлетворить', 'ходатайство'), 9)]


In [58]:
golden_standart


[('принять', 'решение'),
 ('вынести', 'приговор'),
 ('удовлетворить', 'иск'),
 ('возбудить', 'дело'),
 ('проведение', 'следствие'),
 ('наложить', 'арест'),
 ('провести', 'разбирательство'),
 ('выдать', 'санкция'),
 ('продлить', 'срок'),
 ('удовлетворить', 'ходатайство')]

In [59]:
from nltk.metrics.spearman import *
print(list(ranks_from_sequence(l)))

[(('суд', 'признать'), 0), (('выдать', 'санкция'), 1), (('суд', 'удовлетворить'), 2), (('удовлетворить', 'иск'), 3), (('удовлетворить', 'ходатайство'), 4), (('принять', 'решение'), 5), (('санкция', 'арест'), 6), (('наложить', 'арест'), 7), (('суд', 'вынести'), 8), (('суд', 'принять'), 9)]


In [60]:
print('%0.1f' % spearman_correlation(ranks_from_sequence(reversed(golden_standart)), ranks_from_sequence(l)))

-1.9


In [61]:
print('%0.1f' % spearman_correlation(ranks_from_sequence(golden_standart), ranks_from_sequence(golden_standart)))

1.0


In [62]:
print('%0.1f' % spearman_correlation(ranks_from_sequence(golden_standart), ranks_from_sequence(l)))

-3.5


Теперь посмотрим корреляцию золотого стандарта с pmi:

In [63]:
print('%0.1f' % spearman_correlation(ranks_from_sequence(golden_standart), ranks_from_sequence(pmi)))

-6.1


In [64]:
print('%0.1f' % spearman_correlation(ranks_from_sequence(reversed(golden_standart)), ranks_from_sequence(pmi)))

-0.9


Таким образом наиболее качественной, является выборка с частотностью 10 pmi, которая коррелирует с золотым стандартом в обратном порядке с коэффициентом -0.9